# Minimize example - Lammps

In [1]:
# headers

# general modules
import numpy as np
import matplotlib.pyplot as plt

# pyiron modules
from pyiron_atomistics import Project
import pyiron_contrib

In [2]:
# define project

pr = Project('minimize_example')
pr.remove_jobs(recursive=True)

Are you sure you want to delete all jobs from 'minimize_example'? y/(n)y


In [3]:
# check the git head of the repos that this notebook worked on when this notebook was written

pr.get_repository_status()

,Module,Git head
0,pyiron,1b8bc7ff9ead77daf3e0e65ded2eb22e16ac4b69
1,pyiron_mpie,6413178a614cbcffb115052c01517dea65b35d58
2,pyiron_contrib,6216a0d9d6f84ed766243426cec8aa3daacfd193
3,pyiron_base,4f22b9f2f32294d4f9f6f45f1e53b275239d0aed


In [4]:
# inputs 

# structure specific
element = 'Al'
supercell = 3
cubic = True
a = 4.  # we purposefully set this value off the actual a value

# job specific
potential = '2008--Mendelev-M-I--Al--LAMMPS--ipr1'

# NEB specific
n_steps = 100000
f_tol = 1e-4
gamma0 = 0.1

In [5]:
# create base structure

box = pr.create_ase_bulk(name=element, cubic=cubic, a=a).repeat(supercell)

In [6]:
# create and run the minimize job

pr_prot = pr.create_group('protocol_mini')  # create a new folder

mini_ref = pr_prot.create_job(job_type=pr.job_type.Lammps, job_name='ref_mini')
mini_ref.structure = box
mini_ref.potential = potential
mini_ref.save()  # Don't forget this step!

prot_mini_job = pr_prot.create_job(job_type=pr.job_type.ProtoMinimGradDes, job_name='prot_mini_job')
prot_mini_job.input.ref_job_full_path = mini_ref.path
prot_mini_job.input.structure = box
prot_mini_job.input.n_steps = n_steps
prot_mini_job.input.f_tol = f_tol
prot_mini_job.input.gamma0 = gamma0

# set_output_whitelist sets how often an output of a particular vertex is stored in the archive.
# for example, here, the output 'energy_pot' of vertex 'calc_static' is saved every step in the archive.
prot_mini_job.set_output_whitelist(**{'calc_static': {'energy_pot': 1},
                                 'max_force': {'amax': 1}})

prot_mini_job.run()

The job ref_mini was saved and received the ID: 584
The job prot_mini_job was saved and received the ID: 585
The job prot_mini_job_calc_static_job was saved and received the ID: 586


/home/raynol025/pyiron/pyiron_base/pyiron_base/generic/parameters.py:284: UserWarning: The input in GenericParameters changed, while the state of the job was already finished.
  "The input in GenericParameters changed, while the state of the job was already finished."


In [7]:
# check what keys are in the output dictionary

prot_mini_job.output.keys()

dict_keys(['energy_pot', 'max_force', 'positions', 'forces'])

In [8]:
# run pyiron's lammps minimize

pr_pyir = pr.create_group('pyiron_mini')

pyir_mini_job = pr_pyir.create_job(pr.job_type.Lammps, 'pyir_mini_job')
pyir_mini_job.structure = box
pyir_mini_job.potential = potential
pyir_mini_job.calc_minimize(pressure=None, style='sd')
# since this protocol method is a dummy method to demonstrate how a protocol works, it presently can only relax ionic 
# positions, and not the simulation cell. For this reason, we keep pressure=None (isochoric relaxation).

# the protocol uses the gradient descent method for the minimization of forces. We seled the steepest descent method
# for the minimization in the lammps job.
pyir_mini_job.run()

The job pyir_mini_job was saved and received the ID: 587


In [9]:
# compare the energy_pots from the 2 methods

prot_mini_job.output.energy_pot[-1], pyir_mini_job.output.energy_pot[-1]

(-367.82237117900627, -367.822371179006)